In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [ ]:
# Imports
from Models import ModelUtil
import importlib
from Data import Preprocessing
from sklearn.model_selection import train_test_split
from Logs import logging as logs
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

In [ ]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
importlib.reload(logs)
infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering()

In [ ]:
infieldcorrmatrix = infieldDataFrame.corr()
infieldcorrmatrix.to_csv('Infield_Correlation_Matrix')
outfieldcorrmatrix = outfieldDataFrame.corr()
outfieldcorrmatrix.to_csv('Outfield_Correlation_Matrix')

In [1]:
# 2) Training Models
Y = infieldDataFrame["FieldSlice"]
X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "ZoneSpeed", "PlateLocHeight", "PlateLocSide"]]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, random_state=11)

# Here's my KFold code:
## 2) Training Models
#Y = infield[0][['Direction','Distance']]
#X = infield[0][infield[1]]
#trainIn  = []
#trainOut = []
#testIn   = []
#testOut  = []
##display(Y)
##display(X)
#
## KFold:
#kf = KFold(n_splits=X.shape[0], shuffle=True)
#for train_index, test_index in kf.split(X):
#    #print("TRAIN:", train_index, "TEST:", test_index)
#    #xTrain, xTest = X.iloc[train_index,:], X.iloc[test_index,:]
#    #yTrain, yTest = Y.iloc[train_index,:], Y.iloc[test_index,:]
#    trainIn.append(X.iloc[train_index,:])
#    trainOut.append(Y.iloc[train_index,:])
#    testIn.append(X.iloc[test_index,:])
#    testOut.append(Y.iloc[test_index,:])
    

# GroupKFold: (avoids putting data from the same group in the test set -- useful for Pitcher/Batter ID)

#xTrain, xTest, yTrain, yTest = train_test_split(_in, _out, test_size=0.25, random_state=11)

NameError: name 'infieldDataFrame' is not defined

In [ ]:
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 25
max_features = 10
max_leaf_nodes = 55
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)


In [ ]:
# b) Naive Bayes

var_smoothing = 1e-9
result = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)

In [ ]:
# c)Logistic Regression
lr = 0.8
e = 100
result = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)

In [ ]:
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
result = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)

In [ ]:
# z) RandomForestRegressor
for i in range(0, len(trainIn)):
    direction, distance = ModelUtil.runRFR(trainIn[i], trainOut[i], testIn[i], testOut[i])

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements